In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\2101\\OneDrive\\Desktop\\git\\Brain-Tumor-MRI-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path:Path
    updated_base_model_path : Path
    params_image_size : list
    params_learning_rate : float
    params_seed : int
    params_classes : int

In [4]:
from brainTumorMRIClassification.constants import *
from brainTumorMRIClassification.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_seed=self.params.SEED,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [6]:
import os 
import urllib.request as request
import zipfile
from pathlib import Path
from brainTumorMRIClassification import logger
import tensorflow as tf
from brainTumorMRIClassification.utils.common import get_size

In [7]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        from tensorflow.keras import layers, models
        image_shape = tuple(self.config.params_image_size)
        SEED = self.config.params_seed
        N_TYPES = self.config.params_classes

        self.model = models.Sequential([
            # Convolutional layer 1
            layers.Conv2D(32, (4, 4), activation="relu", input_shape=image_shape),
            layers.MaxPooling2D(pool_size=(3, 3)),

            # Convolutional layer 2
            layers.Conv2D(64, (4, 4), activation="relu"),
            layers.MaxPooling2D(pool_size=(3, 3)),

            # Convolutional layer 3
            layers.Conv2D(128, (4, 4), activation="relu"),
            layers.MaxPooling2D(pool_size=(3, 3)),

            # Convolutional layer 4
            layers.Conv2D(128, (4, 4), activation="relu"),
            layers.Flatten(),

            # Fully connected layers
            layers.Dense(512, activation="relu"),
            layers.Dropout(0.4, seed=SEED),
            layers.Dense(N_TYPES, activation="softmax")
        ])

        optimizer = tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate, beta_1=0.869, beta_2=0.995)
        self.model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        self.model.summary()
        self.save_model(path=self.config.base_model_path, model=self.model)

    # def update_base_model(self):
        # For this custom Sequential model, update_base_model can simply save the compiled model again if needed
        # self.save_model(path=self.config.updated_base_model_path, model=self.model)

    @staticmethod
    def save_model(path: Path, model=tf.keras.Model):
        model.save(path)


In [8]:
# Pipeline
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    # prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2025-09-03 12:25:38,959] : INFO : common : YAML file config\config.yaml loaded successfully.
[2025-09-03 12:25:38,964] : INFO : common : YAML file params.yaml loaded successfully.
[2025-09-03 12:25:38,966] : INFO : common : Created directory: artifacts
[2025-09-03 12:25:38,968] : INFO : common : Created directory: artifacts/prepare_base_model


c:\Users\2101\OneDrive\Desktop\git\Brain-Tumor-MRI-Classification\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 147, 147, 32)   │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 46, 46, 64)     │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1, 1, 128)      │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 495,972 (1.89 MB)

 Trainable params: 495,972 (1.89 MB)

 Non-trainable params: 0 (0.00 B)

[2025-09-03 12:25:39,314] : WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
